<a href="https://colab.research.google.com/github/VegetaSan1/Machine-Learning-Projects/blob/main/Tam_Eng_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
#import the necessary libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

#define the batch size for training
batch_size =3 
#define the number of epoch for training
epochs = 150
#define the dimensionality of the encoding process
latent_dim =100
#define the number of samples to train on
num_samples = 150

In [116]:
#define the file location
data_path = 'tam.txt'
#define an empty list to store the words/sentences in English
input_texts = []
#define an empty list to store the characters in Spanish
target_texts = []
#define a set to store the words/sentences in English
#a set data type and not a list is used to avoid repetition of characters
input_characters = set()
#define a set to store the characters in Spanish
target_characters = set()
#read the data file and parse by each line
with open(data_path,encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    #split each line into the input text, target text and the other unnecessary text
    input_text, target_text, _ = line.split('\t')
    #we use tab as the start sequence character for the target, and \n as end sequence character
    target_text = '\t' + target_text + '\n'
    #append the English words for each text to the empty list
    input_texts.append(input_text)
    #append the Spanish words for each text to the empty list
    target_texts.append(target_text)
    #select the letters or words in English
    for char in input_text:
        #check if the letter is not in the list
        if char not in input_characters:
            #append the letter that is not on the list
            input_characters.add(char)
    #select the letters or words in Spanish
    for char in target_text:
        #check if the letter or word is not in the list
        if char not in target_characters:
            #append the letter or word that is not on the list
            target_characters.add(char)

In [117]:
print(input_text)

He bought a pair of shoes.


In [118]:
print(target_text)

	நான் ஒரு ஜோடி காலணிகளை வாங்கினேன்



In [119]:
print(input_texts)

['I slept.', 'Calm down.', "I'll walk.", 'Who is he?', 'Who knows?', 'She smiled.', 'Talk to me!', 'Who is she?', 'Go to sleep.', 'It may rain.', 'She bit him.', 'She hit him.', 'She is kind.', 'She is eight.', 'Where are we?', 'Keep in touch!', 'See you again.', 'Give it to her.', 'I ate too much.', "I'll see to it.", "It's up to you.", 'Leave it to me.', 'Listen to this!', "That's the way.", 'Come and see me.', "Don't lie to me.", 'He began to run.', 'He just arrived.', 'He likes to run.', 'How is your dad?', 'I want to sleep.', "I'm able to run.", 'Raise your hand.', 'What did he say?', 'When can we eat?', 'Come and help us.', 'He is still here.', 'I have to go now.', 'I know that much.', 'I made a mistake.', 'I walk to school.', "That's our house.", 'Those are my CDs.', 'Walk ahead of me.', "We'll follow you.", 'Beware of the dog!', 'He came back soon.', 'He has three sons.', 'I know how to ski.', 'I know what to do.', "I'm kind of happy.", 'Keep to the right.', 'She began to sing.

In [120]:
print(target_texts)

['\tநான் தூங்கினேன்.\n', '\tஅமைதியாக இருங்கள்\n', '\tநான் நடப்பேன்.\n', '\tஅவன் யார்?\n', '\tயாருக்குத் தெரியும்?\n', '\tஅவள் சிரித்தாள்\n', '\tஎன்னிடம் பேசு\n', '\tஅவள் யார்?\n', '\tபோய் தூங்கு\n', '\tமழை பெய்யலாம்\n', '\tஅவள் அவனைக் கடித்தாள்\n', '\tஅவள் அவனைக் அடித்தாள்\n', '\tஅவள் அன்பானவள்\n', '\tஅவளுக்கு எட்டு வயது\n', '\tநாம் எங்கே இருக்கிறோம்?\n', '\tதொடர்பில் இரு\n', '\tமறுபடியும் சந்திப்போம்\n', '\tஅவளிடம் கொடு\n', '\tநான் நிறைய சாப்பிட்டேன்\n', '\tஅதை நான் பார்க்கிறேன்\n', '\tஉன் கையில்தான் இருக்கிறது\n', '\tஎன்னிடம் விட்டுவிடு\n', '\tஇதைக் கேள்\n', '\tஅந்த பக்கம்தான் வழி\n', '\tஎன்னை வந்து பார்\n', '\tஎன்னிடம் பொய் சொல்லாதே\n', '\tஅவன் ஓட ஆரம்பித்தான்\n', '\tஅவன் இப்பொழுதுதான் வந்தான்\n', '\tஅவன் ஓட விருப்பப் படுகிறான்\n', '\tதங்களுடைய தந்தையார் எப்படி இருக்கிறார்கள்?\n', '\tநான் தூங்க விரும்புகிறேன்\n', '\tஎன்னால் ஓட முடிகிறது\n', '\tகையைத் தூக்கு\n', '\tஅவன் என்ன சொன்னான்?\n', '\tஎப்பொழுது நம்மால் சாப்பிட முடியும்\n', '\tவா எங்களுக்கு உதவி செய்\n', '\tஅவன் இன்னும் இருக்கி

In [121]:
print(input_characters)

{'R', 'h', 'p', 'o', 'e', 'k', 'n', "'", 'W', 'K', 'I', 'y', 'j', 'x', '0', '?', 'l', 'm', 'd', 'B', 'i', '2', 'S', 'u', '!', 'v', 'G', 't', 'H', 's', 'b', 'r', 'A', 'N', 'w', 'L', 'D', ',', 'Y', 'P', 'g', 'q', 'c', 'M', 'T', 'C', 'f', ' ', '.', 'a'}


In [122]:
print(target_characters)

{'ல', 'ூ', 'ஷ', 'ு', '(', 'ப', ')', 'ஓ', 'ா', 'ங', 'ம', 'உ', '0', 'ழ', 'ய', 'ை', 'ே', '?', 'ஒ', 'ச', 'வ', 'த', '2', 'ந', '!', 'ள', 'க', 'ன', 'ோ', 'அ', 'ஏ', '\t', 'D', ',', 'எ', 'ர', 'ொ', 'ண', 'ெ', 'ட', 'C', '\n', 'இ', 'ீ', ' ', 'ி', '.', 'ற', 'ஜ', 'ஸ', '்', 'ஆ'}


In [123]:
#define the sorted list of English words
input_characters = sorted(list(input_characters))
#define the sorted list of Spanish words
target_characters = sorted(list(target_characters))
#define the number of English words
num_encoder_tokens = len(input_characters)
#define the number of Spanish words
num_decoder_tokens = len(target_characters)
#define the maximum length of the English sentences
max_encoder_seq_length = max([len(txt) for txt in input_texts])
#define the maximum length of the Spanish sentences
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs", max_decoder_seq_length)

Number of samples: 150
Number of unique input tokens: 50
Number of unique output tokens: 52
Max sequence length for inputs: 26
Max sequence length for outputs 59


In [124]:
#index each characters in English
input_token_index = dict(
[(char, i) for i, char in enumerate(input_characters)])
#index each characters in Spanish
target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters)])

In [125]:
print(input_token_index)

{' ': 0, '!': 1, "'": 2, ',': 3, '.': 4, '0': 5, '2': 6, '?': 7, 'A': 8, 'B': 9, 'C': 10, 'D': 11, 'G': 12, 'H': 13, 'I': 14, 'K': 15, 'L': 16, 'M': 17, 'N': 18, 'P': 19, 'R': 20, 'S': 21, 'T': 22, 'W': 23, 'Y': 24, 'a': 25, 'b': 26, 'c': 27, 'd': 28, 'e': 29, 'f': 30, 'g': 31, 'h': 32, 'i': 33, 'j': 34, 'k': 35, 'l': 36, 'm': 37, 'n': 38, 'o': 39, 'p': 40, 'q': 41, 'r': 42, 's': 43, 't': 44, 'u': 45, 'v': 46, 'w': 47, 'x': 48, 'y': 49}


In [126]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '(': 4, ')': 5, ',': 6, '.': 7, '0': 8, '2': 9, '?': 10, 'C': 11, 'D': 12, 'அ': 13, 'ஆ': 14, 'இ': 15, 'உ': 16, 'எ': 17, 'ஏ': 18, 'ஒ': 19, 'ஓ': 20, 'க': 21, 'ங': 22, 'ச': 23, 'ஜ': 24, 'ட': 25, 'ண': 26, 'த': 27, 'ந': 28, 'ன': 29, 'ப': 30, 'ம': 31, 'ய': 32, 'ர': 33, 'ற': 34, 'ல': 35, 'ள': 36, 'ழ': 37, 'வ': 38, 'ஷ': 39, 'ஸ': 40, 'ா': 41, 'ி': 42, 'ீ': 43, 'ு': 44, 'ூ': 45, 'ெ': 46, 'ே': 47, 'ை': 48, 'ொ': 49, 'ோ': 50, '்': 51}


In [127]:
#define the input data of the encoder as a 3-dimensional matrix populated with zeros
#the shape of the matrix is the length of the input text by the length of the max encoder by the number of encoder characters
#the np.zeros() function takes an argument of the specified dtype. Here we use float32
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens
), dtype='float32')
#define the input data of the decoder as a 3-dimensional matrix populated with zeros
decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')
#define the target data of the decoder as a 3-dimensional matrix populated with zeros
decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')

In [128]:
#parse the input and output texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #define that the decoder_target_data is one time step ahead of the decoder_input_data by 
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # define the decoder_target_data to be ahead by one timestep and will not include the first character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [129]:
encoder_input_data[0].shape

(26, 50)

In [130]:
encoder_input_data.shape

(150, 26, 50)

In [131]:
#define an input of the encoder with length as the number of encoder tokens
encoder_inputs = Input(shape=(None, num_encoder_tokens))
#instantiate the LSTM model
encoder = LSTM(latent_dim, return_state=True)
#define the outputs and states of the encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#disregard encoder_outputs and keep only the states
encoder_states = [state_h, state_c]

In [132]:
#define an input of the encoder with length as the number of encoder tokens
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#define the LSTM model for the decoder setting the return sequences and return state to True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
#define only the decoder output for the training model. The states are only needed in the inference model
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
#define the training model which requires the encoder_input_data and decoder_input_data to return the decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/150
40/40 [==============================] - 6s 55ms/step - loss: 2.5953 - accuracy: 0.5172 - val_loss: 2.3863 - val_accuracy: 0.4661
Epoch 2/150
40/40 [==============================] - 1s 33ms/step - loss: 1.7346 - accuracy: 0.5838 - val_loss: 2.0334 - val_accuracy: 0.4621
Epoch 3/150
40/40 [==============================] - 1s 34ms/step - loss: 1.6140 - accuracy: 0.5888 - val_loss: 1.9661 - val_accuracy: 0.4644
Epoch 4/150
40/40 [==============================] - 1s 34ms/step - loss: 1.5165 - accuracy: 0.5993 - val_loss: 1.9092 - val_accuracy: 0.4785
Epoch 5/150
40/40 [==============================] - 1s 33ms/step - loss: 1.4720 - accuracy: 0.6072 - val_loss: 1.8637 - val_accuracy: 0.4847
Epoch 6/150
40/40 [==============================] - 1s 34ms/step - loss: 1.4351 - accuracy: 0.6165 - val_loss: 1.8275 - val_accuracy: 0.5073
Epoch 7/150
40/40 [==============================] - 1s 34ms/step - loss: 1.4009 - accuracy: 0.6355 - val_loss: 1.7960 - val_accuracy: 0.5237
Epoch 

In [133]:
#define the decoder input state as  a list of the hidden and cell state
encoder_model=Model(encoder_inputs,encoder_states)
decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
#define the decoder output
decoder_outputs = decoder_dense(decoder_outputs)
#define the decoder model
decoder_model = Model(
[decoder_inputs] + decoder_states_inputs, 
    [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequence back to something readable
reverse_input_char_index = dict(
(i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items()
)

In [134]:
def decode_sequence(input_seq):
    #encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    #generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #populate the first character of target sequence with the start character
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    #sampling loop for a batch of sequences
    #to simplify, we use batch size of 1
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value
        )
        
        #sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        #exit condition: either hit max length or find stop character
        if (len(decoded_sentence) > max_decoder_seq_length or sampled_char == '\n'):
            stop_condition = True
            
        #update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        #update states
        states_value = [h, c]
    
    return decoded_sentence

In [135]:
for seq_index in range(10):
    #take one sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #call the function
    decoded_sentence = decode_sequence(input_seq)
    print()
    print(f"Input sentence: {input_texts[seq_index]}")
    print(f"Decoded sentence: {decoded_sentence}")


Input sentence: I slept.
Decoded sentence: நான் தூங்கினேன்.


Input sentence: Calm down.
Decoded sentence: அமைதியாக இருங்கள்


Input sentence: I'll walk.
Decoded sentence: நான் நடப்பேன்.


Input sentence: Who is he?
Decoded sentence: அவன் யார்?


Input sentence: Who knows?
Decoded sentence: யாருக்குத் தெரியும்?


Input sentence: She smiled.
Decoded sentence: அவள் அவனைக் அடித்தாள்


Input sentence: Talk to me!
Decoded sentence: என்னிடம் பேசு


Input sentence: Who is she?
Decoded sentence: அவள் யார்?


Input sentence: Go to sleep.
Decoded sentence: போய் தூங்கு


Input sentence: It may rain.
Decoded sentence: நான் படிக்கி சேரியாம்

